In [1]:
import sys
import pandas as pd
import os
import numpy as np
import os.path
from pprint import pprint
import importlib.util


In [2]:
PHEN_START = 0
PHEN_END = 12
PHEN_SPECS_FILE = '/cs/labs/michall/roeizucker/10krun/phenotype_specs.py'
WORK_PATH = '/cs/labs/michall/roeizucker/10krun/runs/'
PHEN_PER_DIR = 12

In [3]:
spec = importlib.util.spec_from_file_location("module.name", "/cs/labs/michall/roeizucker/10krun/phenotype_specs.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
old_specs = foo.specs
new_specs = []
for spec in old_specs:
#     print(spec)
    new_specs.append(spec)
    spec_f = spec.copy()
    spec_f['name']+="_F"
    spec_f['sex_filter'] = 'F'
    spec_m = spec.copy()
    spec_m['name']+="_M"
    spec_m['sex_filter'] = 'M'
    new_specs.append(spec_f)
    new_specs.append(spec_m)


In [5]:
print(len(new_specs))

294


# For Rerun

In [6]:
spec = importlib.util.spec_from_file_location("module.name", "/cs/labs/michall/roeizucker/10krun/phenotype_specs.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
old_specs = foo.specs
new_specs = []
for spec in old_specs:
#     print(spec)
    new_specs.append(spec)
    spec_f = spec.copy()
    spec_f['name']+="_F"
    spec_f['sex_filter'] = 'F'
    spec_m = spec.copy()
    spec_m['name']+="_M"
    spec_m['sex_filter'] = 'M'
    new_specs.append(spec_f)
    new_specs.append(spec_m)


In [ ]:
# creates the scripts and folders
counter = 0
# print(new_specs)
while counter < len(new_specs):
#     create dir for current batch of phenotypes
    curr_counter = counter
    dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
#     print(dir_name)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
#     print(dir_name)
    curr_phenotypes = []
    for i in range(counter,counter + PHEN_PER_DIR):
        if i >= len(new_specs):
            break
        phen = new_specs[i]
        curr_phenotypes.append(phen)
#         create dir for phenotype_raw_results
        phenotype_raw_results = dir_name + "/" + phen["name"] + "_1"
        if not os.path.exists(phenotype_raw_results):
            os.mkdir(phenotype_raw_results)
            
#         create script_to run phenotype
#             activation of the next batch, in this next add 3 instead of 1, becuase we rerun only the the non gendered.
        if i < len(new_specs) - 4:
            next_phen =  new_specs[i+3]
#             if moves to the next dir
            new_dir_name = dir_name
            if i%PHEN_PER_DIR + 3 >= PHEN_PER_DIR:
                new_dir_name = WORK_PATH + str(curr_counter + PHEN_PER_DIR) + ":" + str(curr_counter + PHEN_PER_DIR + PHEN_PER_DIR - 1)
                
            new_phenotype_raw_results = new_dir_name + "/" + next_phen["name"] + "_1"
        command ="source ~/my_python/bin/activate\nif test $SLURM_ARRAY_TASK_ID -eq 250\nthen\n\tFILE="+new_dir_name+"/"+next_phen["name"]+"_flag\n\tif ! test -f \"$FILE\"\n\tthen\n\t\ttouch "+new_dir_name+"/"+next_phen["name"]+"_flag" +"\n\t\tsbatch --array=0-300 --mem=30g -c1 --time=1-0 --killable --requeue --wrap=\""+ new_dir_name + "/run_"+ next_phen["name"]+ "_1.sh\"\n\tfi\nfi\n"
        command += "pwas_test_genes --resolve-quasi-complete-covariate-separation --remove-multicollinear-covariate --dataset-file=" + dir_name + "/ukbb_dataset.csv --gene-effect-scores-dir=/cs/labs/michall/roeizucker/virt_env_install_test/ukbb_imputation_gene_effect_scores/ --per-gene-pwas-results-dir="+phenotype_raw_results+" --sample-id-col=eid --phenotype-col='" + phen["name"] + "' --covariate-cols-json-file=" + dir_name + "/ukbb_covariate_columns.json --task-index-env-variable=SLURM_ARRAY_TASK_ID --total-tasks-env-variable=SLURM_ARRAY_TASK_COUNT\n"

        run_script_path = dir_name + "/run_"+ phen["name"] +"_1.sh"
        !rm -f {run_script_path}
        if not os.path.exists(run_script_path) and not os.path.exists('/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'_1.csv'):
            with open(run_script_path, 'w') as fp:
    #         create run file
                fp.write(command)
    #         give run privlige to script
            os.system("chmod 744 " + run_script_path)
        counter+=1    
#     create phenotype_specs file
    if not os.path.exists(dir_name + "/phenotype_specs.py"):
        with open(dir_name + "/phenotype_specs.py", 'w') as fp:
            fp.write("specs = "+ str(curr_phenotypes))
        
#     create script to run all scripts
# TODO: remove all phenotypes that were already run
    run_all_script_path = dir_name + "/run_all.sh"
    if not os.path.exists(run_all_script_path):
        with open(run_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+ "_1" +'.csv'):
                    fp.write('sbatch --array=0-300 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="' + dir_name + "/run_"+ phen["name"]+ '.sh"\n')
                    fp.write('sleep .5\n')
        os.system("chmod 744 " + run_all_script_path)
    
#   create script to combine all phenotype results
# TODO: remove all phenotypes that were already run
    combine_all_script_path = dir_name + "/combine_all_1.sh"
    if not os.path.exists(combine_all_script_path):
        with open(combine_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/10krun/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'_1.csv'):
                    fp.write('combine_pwas_results --genes-file=~/my_storage/virt_env_install_test/genes_hg19.csv --per-gene-pwas-results-dir='+dir_name + "/" + phen["name"]+'_1/ --results-file=/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'_1.csv\n')
        os.system("chmod 744 " + combine_all_script_path)

print("Done")

^C
/usr/bin/lua: interrupted!
stack traceback:
	[C]: in ?
	[C]: in function 'require'
	/usr/share/lmod/lmod/libexec/../tools/Optiks.lua:64: in main chunk
	[C]: in function 'require'
	/usr/share/lmod/lmod/libexec/Master.lua:67: in main chunk
	[C]: in function 'require'
	/usr/share/lmod/lmod/libexec/lmod:128: in main chunk
	[C]: in ?
^C
/usr/bin/lua: interrupted!
stack traceback:
	[C]: in ?
	[C]: in function 'require'
	/usr/share/lua/5.2/posix/init.lua:29: in main chunk
	[C]: in function 'require'
	/usr/share/lmod/lmod/libexec/lmod:64: in main chunk
	[C]: in ?
^C
^C




In [ ]:
counter = 0
# print(new_specs)
commands = []
while counter < len(new_specs):
    combine_all_script_path = WORK_PATH + "/combine_all_1.sh"
    with open(combine_all_script_path, 'w') as fp:
    #     create dir for current batch of phenotypes
        curr_counter = counter
        dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
    #     print(dir_name)
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
    #     print(dir_name)
        curr_phenotypes = []
        for i in range(counter,counter + PHEN_PER_DIR):
            if i >= len(new_specs):
                break
            phen = new_specs[i]
            curr_phenotypes.append(phen)
            os.system('combine_pwas_results --genes-file=~/my_storage/virt_env_install_test/genes_hg19.csv --per-gene-pwas-results-dir='+dir_name + "/" + phen["name"]+'_1/ --results-file=/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'_1.csv\n')
            counter+=1  
#     print(commands)

# regular

In [23]:
# creates the scripts and folders
counter = 0
while counter < len(new_specs):
#     create dir for current batch of phenotypes
    dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
#     print(dir_name)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
#     print(dir_name)
    curr_phenotypes = []
    for i in range(counter,counter + PHEN_PER_DIR):
        if i >= len(new_specs):
            break
        phen = new_specs[i]
        curr_phenotypes.append(phen)
#         create dir for phenotype_raw_results
        phenotype_raw_results = dir_name + "/" + phen["name"]
        if not os.path.exists(phenotype_raw_results):
            os.mkdir(phenotype_raw_results)
            
#         create script_to run phenotype
        command = "source ~/my_python/bin/activate\npwas_test_genes --resolve-quasi-complete-covariate-separation --remove-multicollinear-covariate --dataset-file=" + dir_name + "/ukbb_dataset.csv --gene-effect-scores-dir=/cs/labs/michall/roeizucker/virt_env_install_test/ukbb_imputation_gene_effect_scores/ --per-gene-pwas-results-dir="+phenotype_raw_results+" --sample-id-col=eid --phenotype-col='" + phen["name"] + "' --covariate-cols-json-file=" + dir_name + "/ukbb_covariate_columns.json --task-index-env-variable=SLURM_ARRAY_TASK_ID --total-tasks-env-variable=SLURM_ARRAY_TASK_COUNT\n"
        run_script_path = dir_name + "/run_"+ phen["name"] +"_1.sh"

        if not os.path.exists(run_script_path) and not os.path.exists('/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'.csv'):
            with open(run_script_path, 'w') as fp:
    #         create run file
                fp.write(command)
    #         give run privlige to script
            os.system("chmod 744 " + run_script_path)
        counter+=1    
#     create phenotype_specs file
    if not os.path.exists(dir_name + "/phenotype_specs.py"):
        with open(dir_name + "/phenotype_specs.py", 'w') as fp:
            fp.write("specs = "+ str(curr_phenotypes))
        
#     create script to run all scripts
# TODO: remove all phenotypes that were already run
    run_all_script_path = dir_name + "/run_all.sh"
    if not os.path.exists(run_all_script_path):
        with open(run_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'.csv'):
                    fp.write('sbatch --array=0-300 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="' + dir_name + "/run_"+ phen["name"]+ '.sh"\n')
                    fp.write('sleep .5\n')
        os.system("chmod 744 " + run_all_script_path)
    
#   create script to combine all phenotype results
# TODO: remove all phenotypes that were already run
    combine_all_script_path = dir_name + "/combine_all.sh"
    if not os.path.exists(combine_all_script_path):
        with open(combine_all_script_path, 'w') as fp:
            for phen in curr_phenotypes:
                if not os.path.exists('/cs/labs/michall/roeizucker/10krun/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'.csv'):
                    fp.write('combine_pwas_results --genes-file=~/my_storage/virt_env_install_test/genes_hg19.csv --per-gene-pwas-results-dir='+dir_name + "/" + phen["name"]+'/ --results-file=/cs/labs/michall/roeizucker/10krun/results/'+phen["name"]+'.csv\n')
        os.system("chmod 744 " + combine_all_script_path)
print("Done")

Done


In [ ]:
counter = 0
while counter < len(new_specs):
#     create dir for current batch of phenotypes
#     print(str(counter) + ":" + str(counter + PHEN_PER_DIR - 1))
    dir_name = WORK_PATH + str(counter) + ":" + str(counter + PHEN_PER_DIR - 1)
    file_name = dir_name + "/ukbb_dataset.csv"
    print(dir_name)
    print(file_name)
    if not os.path.exists(file_name ):
        os.system('create_ukbb_phenotype_dataset --phenotype-specs-file=' +dir_name+'/phenotype_specs.py --output-dataset-file='+file_name+' --output-covariates-columns-file=' + dir_name +'/ukbb_covariate_columns.json')
    counter += PHEN_PER_DIR

/cs/labs/michall/roeizucker/10krun/runs/0:11
/cs/labs/michall/roeizucker/10krun/runs/0:11/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/12:23
/cs/labs/michall/roeizucker/10krun/runs/12:23/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/24:35
/cs/labs/michall/roeizucker/10krun/runs/24:35/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/36:47
/cs/labs/michall/roeizucker/10krun/runs/36:47/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/48:59
/cs/labs/michall/roeizucker/10krun/runs/48:59/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/60:71
/cs/labs/michall/roeizucker/10krun/runs/60:71/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/72:83
/cs/labs/michall/roeizucker/10krun/runs/72:83/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/84:95
/cs/labs/michall/roeizucker/10krun/runs/84:95/ukbb_dataset.csv
/cs/labs/michall/roeizucker/10krun/runs/96:107
/cs/labs/michall/roeizucker/10krun/runs/96:107/ukbb_dataset.csv
/cs/labs/michall/ro

In [69]:
file_set = set()
file_list = []
dataset = {}

In [70]:
# smart runs
import os
for i in range(0,int(len(new_specs) / PHEN_PER_DIR)):
    curr_path = f'{WORK_PATH}{i*PHEN_PER_DIR}:{i*PHEN_PER_DIR + PHEN_PER_DIR - 1}'
#     print(curr_path)
    first = True
    print(i)
    for x in os.walk(curr_path):
        if first:
            first = False
            continue
        num_of_files = len([name for name in os.listdir(x[0] + '/')])
        with open(curr_path + '/ukbb_dataset.csv') as f:
            first_line = f.readline()
        i = 0
        val_to_check = ''
        for val in first_line.split(','):
            i+=1
            if x[0] == curr_path +'/'+ val:
                val_to_check = val
                break
        f = os.popen('cat '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep 1 | wc -l')
        now = f.read()
        num_of_cases = int(now)
        if num_of_files != 17924 and num_of_cases >= 130 and not os.path.exists('/cs/labs/michall/roeizucker/10krun/results/'+val+'.csv'):
            if x[0] not in file_set:
                file_set.add(x[0])
                file_list.append(x[0])
                dataset[x[0]] = {}
                dataset[x[0]]['name'] = x[0]
                dataset[x[0]]['num_of_cases'] = num_of_cases
        
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [75]:
print("#!/bin/bash\n\nsmart_wait () {\nNUM=`squeue | grep roeizuck | grep '\[' | wc -l`\nwhile [ $NUM -gt 4 ]\ndo\n        NUM=`squeue | grep roeizuck | grep '\[' | wc -l`\n        sleep 10\ndone\n}")
counter = 0
for file in file_list:
#     print(file)
    counter += 1
    todo = f"sbatch --array=0-249 --mem=30g -c1 --time=1-0 --killable --requeue --wrap=\"{file}\""
#     todo = todo.replace("Phecode", "run_Phecode")[0:-1] + ".sh\""
    position = todo.rfind("/")
    todo = todo[0:position + 1] + "run_" + todo[position+1:-1] + ".sh\""
    print("cd " +file[0:file.rfind("/")])
    print(todo)
    print("echo \"" + file + "\"")
    print(f"echo \"{counter} / {len(file_list)}\"")
    print("sleep 60")
    print("smart_wait")

#!/bin/bash

smart_wait () {
NUM=`squeue | grep roeizuck | grep '\[' | wc -l`
while [ $NUM -gt 4 ]
do
        NUM=`squeue | grep roeizuck | grep '\[' | wc -l`
        sleep 10
done
}
cd /cs/labs/michall/roeizucker/10krun/runs/12:23
sbatch --array=0-250 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/10krun/runs/12:23/run_D649_M.sh"
echo "/cs/labs/michall/roeizucker/10krun/runs/12:23/D649_M"
echo "1 / 82"
sleep 60
smart_wait
cd /cs/labs/michall/roeizucker/10krun/runs/12:23
sbatch --array=0-250 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/10krun/runs/12:23/run_E11_F.sh"
echo "/cs/labs/michall/roeizucker/10krun/runs/12:23/E11_F"
echo "2 / 82"
sleep 60
smart_wait
cd /cs/labs/michall/roeizucker/10krun/runs/12:23
sbatch --array=0-250 --mem=30g -c1 --time=1-0 --killable --requeue --wrap="/cs/labs/michall/roeizucker/10krun/runs/12:23/run_E03_M.sh"
echo "/cs/labs/michall/roeizucker/10krun/runs/12:23/E03_M"
echo "3 / 82"
slee

In [77]:
print(len(file_set))

82


In [98]:
already_shown = set()

In [9]:
code_translator = pd.read_csv("/cs/labs/michall/roeizucker/10krun/ICD_code_translation.csv")
# print(code_translator.set_index("coding").T.to_dict('list'))
code_translator = code_translator.set_index("coding").T.to_dict('list')


# unneeded manual tranlation
# code_translation = {}
# code_translation["D12"] = "Benign neoplasm of colon, rectum, anus and anal canal"
# code_translation["C50"] = "Malignant neoplasms of breast"
# code_translation["D64"] = "Other anemias"
# code_translation["C44"] = "Other and unspecified malignant neoplasm of skin"
# code_translation["E03"] = "Other hypothyroidism"
# code_translation["E11"] = "Type 2 diabetes mellitus"
# code_translation["E039"] = "Hypothyroidism, unspecified"
# code_translation["D649"] = "Anemia, unspecified"
# code_translation["E66"] = "Overweight and obesity"
# code_translation["E119"] = "Type 2 diabetes mellitus without complications"
# code_translation["E78"] = "Disorders of lipoprotein metabolism and other lipidemias"
# code_translation["F17"] = "Nicotine dependence"
# code_translation["F171"] = "Mental and behavioural disorders due to use of tobacco, harmful use" # TODO: this is from another source
# code_translation["E669"] = "Obesity, unspecified"
# code_translation["F32"] = "Depressive episode"
# code_translation["E780"] = "Pure hypercholesterolemia"
# code_translation["H269"] = "Unspecified cataract"
# code_translation["H26"] = "Other cataract"
# code_translation["I10"] = "Essential (primary) hypertension"
# code_translation["F329"] = "Major depressive disorder, single episode, unspecified"
# code_translation["I209"] = "Angina pectoris, unspecified"
# code_translation["I251"] = "Atherosclerotic heart disease of native coronary artery"
# code_translation["I25"] = "Chronic ischemic heart disease"
# code_translation["I20"] = "Angina pectoris"
# code_translation["I849"] = "Not found, delete this" # delete this
# code_translation["I48"] = "Atrial fibrillation and flutter"
# code_translation["I259"] = "Chronic ischemic heart disease, unspecified"
# code_translation["K21"] = "Gastro-esophageal reflux disease"
# code_translation["J18"] = "Pneumonia, unspecified organism"
# code_translation["J45"] = "Asthma"
# code_translation["J459"] = "J459"


In [7]:
def get_relevant_genes_num(fname,rate = None):
    with open(fname, 'r') as read_obj:
        counter = 0
        read_obj.readline()
        for line in read_obj:
            if rate is None and line.split(',')[-2] == 'True':
                counter+=1
            elif rate is not None and float(line.split(',')[-1]) <= rate:
                counter+=1
            else:
                return counter

    return -1

In [5]:
def get_num_for_binary_variable(curr_path, i,is_case):
    value_to_search = "0.0"
    if is_case:
        value_to_search = "1.0"
    f = os.popen('tail -n +2 '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep '+value_to_search+' | wc -l')
    now = f.read()
    num_of_appearances = int(now)
    f.close()
    return num_of_appearances

def create_phenotype_dict(phenotype_name,num_of_cases,num_of_controls,result_file_path):
    phenotype = {}
    phenotype["prevalence"] = num_of_cases / (num_of_cases + num_of_controls)
    phenotype["name"] = phenotype_name
    phenotype["num_of_cases"] = num_of_cases
    phenotype["num_of_controls"] = num_of_controls
    phenotype["num_relevant_genes"] = -1
    phenotype["num_relevant_genes_0.1"] = -1 # added
    phenotype["result_file_path"] = ""
    if os.path.isfile(result_file_path):
        phenotype["num_relevant_genes"] = get_relevant_genes_num(result_file_path)
        phenotype["num_relevant_genes_0.1"] = get_relevant_genes_num(result_file_path,0.1) # added
        phenotype["result_file_path"] = result_file_path
    phenotype["sex"] = "B"
    if phenotype["name"][-1] == 'F':
        phenotype["sex"] = 'F'
    if phenotype["name"][-1] == 'M':
        phenotype["sex"] = 'M'
    
    return phenotype


output = []
# added
print("name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio") 
for i in range(0,25):
    if i == 462:
        continue
    curr_path = f'/cs/labs/michall/roeizucker/10krun/runs/{i*PHEN_PER_DIR}:{i*PHEN_PER_DIR + PHEN_PER_DIR - 1}'
    first = True
#     iterate for each directory, each directory represents a phenotype
    for x in os.walk(curr_path):
        if first:
            first = False
            continue
        if '/cs/labs/michall/roeizucker/10krun/runs/0:9/plink_run' in x[0]:
            continue
        if "_1" in x[0] or "__pycache__" in x[0] or "_2" in x[0]:
            continue
        num_of_files = len([name for name in os.listdir(x[0] + '/')])
        with open(curr_path + '/ukbb_dataset.csv') as f:
            first_line = f.readline()
        i = 0
        val_to_check = ''
        for val in first_line.split(','):
            i+=1
            if x[0] == curr_path +'/'+ val:
                val_to_check = val
                break
#         Find number of cases
        if "GWAS" in x[0]:
            continue
        num_of_cases = get_num_for_binary_variable(curr_path, i,True)
        num_of_controls = get_num_for_binary_variable(curr_path, i,False)        
        phenotype_name = val_to_check
        path = x[0]        
        result_file_path = f'/cs/labs/michall/roeizucker/10krun/results/{phenotype_name}.csv'
        phenotype = create_phenotype_dict(phenotype_name,num_of_cases,num_of_controls,result_file_path)
        key = phenotype['name']
        icd_code = phenotype_name.split("_")[0]
        icd_list = str([icd_code])
#         output the data
        display_name = code_translator[icd_code][1]
#         prevalence,MF_prevalence_ration,FM_prevalence_ration
        val = f"{phenotype['name'].replace(',',';')},{phenotype['num_of_cases']},{phenotype['num_of_controls']},{phenotype['num_relevant_genes']},{phenotype['num_relevant_genes_0.1']},{phenotype['result_file_path']},{phenotype['sex']},{display_name.replace(',',';') } ,{icd_list},Binary,10krun,{curr_path }/ukbb_dataset.csv,{i},{phenotype['prevalence']},,"
        print(val, flush=True)
        if val not in output:
            output.append(val)        


name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio


KeyboardInterrupt: 

In [121]:
phen_dict = {}
print(list(enumerate("name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio".split(",")) ))
for val in output:
    vals = val.split(",")
    if vals[0][-2] == "_":
        if vals[0][:-2] not in phen_dict:
            phen_dict[vals[0][:-2]] = {}
        if vals[0][-1] == "F":
            phen_dict[vals[0][:-2]]["F"] = vals[13]
        else:
            phen_dict[vals[0][:-2]]["M"] = vals[13]
            
    else:
        if vals[0] not in phen_dict:
            phen_dict[vals[0]] = {}
        phen_dict[vals[0]]["reg"] = vals[13]


[(0, 'name'), (1, 'num_of_cases'), (2, 'num_of_controls'), (3, 'num_relevant_genes'), (4, 'num_relevant_genes_0.1'), (5, 'result_file_path'), (6, 'sex'), (7, 'description'), (8, 'icd_codes'), (9, 'data_type'), (10, 'source'), (11, 'dataset_path'), (12, 'dataset_column'), (13, 'prevalence'), (14, 'MF_prevalence_ratio'), (15, 'FM_prevalence_ratio')]


NameError: name 'output' is not defined

In [96]:
phen_dict

{'D12': {'M': '0.05995654284827405',
  'F': '0.036391545953627424',
  'reg': '0.04716473088231013'},
 'C50': {'M': '0.0006128572679300547',
  'reg': '0.07160333281942312',
  'F': '0.07160333281942312'},
 'D64': {'reg': '0.041441067737897706',
  'F': '0.043255598828284725',
  'M': '0.039286538629905846'},
 'C44': {'reg': '0.06893138881611505',
  'F': '0.06023474524577197',
  'M': '0.07925756719542187'},
 'E03': {'F': '0.07454602736236703',
  'reg': '0.049802782871947136',
  'M': '0.020423269474136627'},
 'D649': {'M': '0.03915123247984336',
  'reg': '0.04132462958111373',
  'F': '0.04315505117875361'},
 'E11': {'F': '0.04552127253105247',
  'reg': '0.06509256833464326',
  'M': '0.0883310384349058'},
 'E039': {'M': '0.02035163680645649',
  'reg': '0.04967178994556516',
  'F': '0.07436504159321103'},
 'E66': {'M': '0.045048988785507914',
  'F': '0.04550116300114624',
  'reg': '0.04529444298896748'},
 'E119': {'F': '0.044917986633865786',
  'M': '0.08704165041666335',
  'reg': '0.064175617

In [97]:
new_output = []
for val in output:
    vals = val.split(",")
    name = vals[0]
    if vals[0][-2] == "_":
        name = vals[0][:-2]
    mf_ratio = float(phen_dict[name]["M"]) / float(phen_dict[name]["F"])
    if mf_ratio == 0:
        fm_ratio = "inf"
    else:
        fm_ratio = 1 / mf_ratio
    new_output.append(val[:-1] + str(mf_ratio) + "," + str(fm_ratio))

In [82]:
new_output

["D12_M,11389,118108,4,5,/cs/labs/michall/roeizucker/10krun/results/D12_M.csv,M,D12 Benign neoplasm of colon ,['D12'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/ukbb_dataset.csv,11,0.08794798335096565,1.1980431979119208,0.8346944431911204",
 "C50_M,11389,125564,-1,-1,,M,C50 Malignant neoplasm of breast ,['C50'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/ukbb_dataset.csv,8,0.0831599161756223,1.0944630639708512,0.913690039362199",
 "D64,11389,263435,0,4,/cs/labs/michall/roeizucker/10krun/results/D64.csv,B,D64 Other anaemias ,['D64'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/ukbb_dataset.csv,12,0.041441067737897706,1.1667373234212,0.857090949201591",
 "C44,11389,255880,47,65,/cs/labs/michall/roeizucker/10krun/results/C44.csv,B,C44 Other malignant neoplasms of skin ,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/ukbb_dataset.csv,3,0.04261249901784345,1.192914253336691,0.8382832187669046",
 "C44_F,11389,140197,8,18,/cs/lab

In [98]:
with open('/cs/labs/michall/roeizucker/phewas_run/discription9.csv', 'a') as the_file:
    the_file.write("name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio\n")
    for line in new_output:
#         TODO: remove '"' in the begining of phenotype name and at the end of dataset_column
        the_file.write(line + "\n")

In [3]:
print(pd.read_csv('/cs/labs/michall/roeizucker/phewas_run/discription9.csv').sort_values("name"))

       name  num_of_cases  num_of_controls  num_relevant_genes  \
3       C44         18944           255880                  47   
4     C44_F          8986           140197                   8   
10    C44_M          9958           115683                  11   
6       C50         10682           138501                   3   
8     C50_F         10682           138501                   2   
..      ...           ...              ...                 ...   
292    Z966         11054           263770                   0   
293  Z966_F          6043           143140                  -1   
290  Z966_M          5011           120630                  -1   
288   Z96_F         10189           138994                  -1   
291   Z96_M          8918           116723                  -1   

     num_relevant_genes_0.1  \
3                        65   
4                        18   
10                       19   
6                         5   
8                         3   
..                   

In [157]:
for spec in new_specs:
    contained = False
    for out in output:
        if spec['name'] in out:
            contained = True
    if not contained:
        print(spec['name'])

### combine with old discription 

In [115]:
df = (pd.read_csv('/cs/labs/michall/roeizucker/phewas_run/discription9.csv').sort_values("name"))
df_old = (pd.read_csv('/cs/labs/michall/roeizucker/phewas_run/discription8.csv').sort_values("name"))

In [116]:
for index, row in df.iterrows():
    df_old.loc[df_old['name'] == row['name'], ['num_of_cases']] = row['num_of_cases']


In [118]:
filtered_df = df_old.drop_duplicates(subset=['icd_codes','sex'], keep="first")


In [117]:
# df["num_of_cases"].dropna()
df_old

,name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio
3,C44,18944,255880,47,65,/cs/labs/michall/roeizucker/10krun/results/C44...,B,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,3,0.042612,1.192914,0.838283
4,C44_F,8986,140197,8,18,/cs/labs/michall/roeizucker/10krun/results/C44...,F,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,4,0.075132,1.192914,0.838283
10,C44_M,9958,115683,11,19,/cs/labs/michall/roeizucker/10krun/results/C44...,M,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,5,0.089626,1.192914,0.838283
6,C50,10682,138501,3,5,/cs/labs/michall/roeizucker/10krun/results/C50...,B,C50 Malignant neoplasm of breast,['C50'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,6,0.075982,1.094463,0.913690
8,C50_F,10682,138501,2,3,/cs/labs/michall/roeizucker/10krun/results/C50...,F,C50 Malignant neoplasm of breast,['C50'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,7,0.075982,1.094463,0.913690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Z966,11054,263770,0,0,/cs/labs/michall/roeizucker/10krun/results/Z96...,B,Z96.6 Presence of orthopaedic joint implants,['Z966'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,6,0.041391,1.170506,0.854332
293,Z966_F,6043,143140,-1,-1,NaN,F,Z96.6 Presence of orthopaedic joint implants,['Z966'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,7,0.073701,1.170506,0.854332
290,Z966_M,5011,120630,-1,-1,NaN,M,Z96.6 Presence of orthopaedic joint implants,['Z966'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,8,0.086268,1.170506,0.854332
288,Z96_F,10189,138994,-1,-1,NaN,F,Z96 Presence of other functional implants,['Z96'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,4,0.075733,1.173840,0.851905


In [119]:
filtered_df.to_csv('/cs/labs/michall/roeizucker/phewas_run/discription9_no_duplicates.csv')

In [113]:
filtered_df[filtered_df["num_of_cases"].isna()]

,name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,prevalence,MF_prevalence_ratio,FM_prevalence_ratio


## create discription correctly (old implementation was stupid)

In [60]:
def get_num_for_binary_variable(curr_path, i,is_case):
    value_to_search = "0.0"
    if is_case:
        value_to_search = "1.0"
    f = os.popen('tail -n +2 '+ curr_path +'/ukbb_dataset.csv | awk -F , \'{print $' + str(i) +'}\' | grep '+value_to_search+' | wc -l')
    now = f.read()
    num_of_appearances = int(now)
    f.close()
    return num_of_appearances

def create_phenotype_dict(phenotype_name,num_of_cases,num_of_controls,result_file_path):
    phenotype = {}
    phenotype["prevalence"] = num_of_cases / (num_of_cases + num_of_controls)
    phenotype["name"] = phenotype_name
    phenotype["num_of_cases"] = num_of_cases
    phenotype["num_of_controls"] = num_of_controls
    phenotype["num_relevant_genes"] = -1
    phenotype["num_relevant_genes_0.1"] = -1 # added
    phenotype["result_file_path"] = ""
    if os.path.isfile(result_file_path):
        phenotype["num_relevant_genes"] = get_relevant_genes_num(result_file_path)
        phenotype["num_relevant_genes_0.1"] = get_relevant_genes_num(result_file_path,0.1) # added
        phenotype["result_file_path"] = result_file_path
    phenotype["sex"] = "B"
    if phenotype["name"][-1] == 'F':
        phenotype["sex"] = 'F'
    if phenotype["name"][-1] == 'M':
        phenotype["sex"] = 'M'
    
    return phenotype
output = []
# added
cols = "name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column".split(",")

for i in range(0,25):
    if i == 462:
        continue
    curr_path = f'/cs/labs/michall/roeizucker/10krun/runs/{i*PHEN_PER_DIR}:{i*PHEN_PER_DIR + PHEN_PER_DIR - 1}'
    first = True
#     iterate for each directory, each directory represents a phenotype
    for x in os.walk(curr_path):
        if first:
            first = False
            continue
        if '/cs/labs/michall/roeizucker/10krun/runs/0:9/plink_run' in x[0]:
            continue
        if "_1" in x[0] or "__pycache__" in x[0] or "_2" in x[0]:
            continue
        num_of_files = len([name for name in os.listdir(x[0] + '/')])
        with open(curr_path + '/ukbb_dataset.csv') as f:
            first_line = f.readline()
        i = 0
        val_to_check = ''
        for val in first_line.split(','):
            i+=1
            if x[0] == curr_path +'/'+ val:
                val_to_check = val
                break
#         Find number of cases
        if "GWAS" in x[0]:
            continue
        num_of_cases = get_num_for_binary_variable(curr_path, i,True)
        num_of_controls = get_num_for_binary_variable(curr_path, i,False)        
        phenotype_name = val_to_check
        path = x[0]        
        result_file_path = f'/cs/labs/michall/roeizucker/10krun/results/{phenotype_name}.csv'
        phenotype = create_phenotype_dict(phenotype_name,num_of_cases,num_of_controls,result_file_path)
        key = phenotype['name']
        icd_code = phenotype_name.split("_")[0]
        icd_list = str([icd_code])
#         output the data
        display_name = code_translator[icd_code][1]
#         prevalence,MF_prevalence_ration,FM_prevalence_ration
        val = f"{phenotype['name'].replace(',',';')},{phenotype['num_of_cases']},{phenotype['num_of_controls']},{phenotype['num_relevant_genes']},{phenotype['num_relevant_genes_0.1']},{phenotype['result_file_path']},{phenotype['sex']},{display_name.replace(',',';') } ,{icd_list},Binary,10krun,{curr_path }/ukbb_dataset.csv,{i}".split(",")
        if val not in output:
            output.append(val)        


In [73]:
df = pd.DataFrame(output, columns=cols)

,name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column
0,D12_M,7533,118108,4,5,/cs/labs/michall/roeizucker/10krun/results/D12...,M,D12 Benign neoplasm of colon,['D12'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,11
1,C50_M,77,125564,-1,-1,,M,C50 Malignant neoplasm of breast,['C50'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,8
2,D64,11389,263435,0,4,/cs/labs/michall/roeizucker/10krun/results/D64...,B,D64 Other anaemias,['D64'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,12
3,C44,18944,255880,47,65,/cs/labs/michall/roeizucker/10krun/results/C44...,B,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,3
4,C44_F,8986,140197,8,18,/cs/labs/michall/roeizucker/10krun/results/C44...,F,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Z96,19107,255717,0,0,/cs/labs/michall/roeizucker/10krun/results/Z96...,B,Z96 Presence of other functional implants,['Z96'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,3
290,Z966_M,5011,120630,-1,-1,,M,Z96.6 Presence of orthopaedic joint implants,['Z966'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,8
291,Z96_M,8918,116723,-1,-1,,M,Z96 Presence of other functional implants,['Z96'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,5
292,Z966,11054,263770,0,0,/cs/labs/michall/roeizucker/10krun/results/Z96...,B,Z96.6 Presence of orthopaedic joint implants,['Z966'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/288:29...,6


In [62]:
df["overall_number_of_participants"] = df.num_of_cases.astype(int) + df.num_of_controls.astype(int)
df["prevalence"] = df.num_of_cases.astype(int) / (df.num_of_cases.astype(int) + df.num_of_controls.astype(int))
df["MF_prevalence_ratio"] = -1.0
df["FM_prevalence_ratio"] = -1.0
prevalence_dict = {}
for i,name in enumerate(df["name"]):
    sexless_name = name
    if "_" in name:
        sexless_name = name[:name.find("_")]
    m_result_list = list(df[df["name"] == sexless_name + "_M"]["prevalence"])
    f_result_list = list(df[df["name"] == sexless_name + "_F"]["prevalence"])
    if len(m_result_list)  == 0 or len(f_result_list)  == 0:
        continue
    M_prevalence = m_result_list[0]
    F_prevalence = f_result_list[0]
    if F_prevalence > 0:
        df.at[i,"MF_prevalence_ratio"] = M_prevalence/F_prevalence
    if M_prevalence > 0:
        df.at[i,"FM_prevalence_ratio"] = F_prevalence/M_prevalence

In [68]:
df.to_csv("/cs/labs/michall/roeizucker/phewas_run/discription10_partial.csv",index=False)

In [74]:
df

,name,num_of_cases,num_of_controls,num_relevant_genes,num_relevant_genes_0.1,result_file_path,sex,description,icd_codes,data_type,source,dataset_path,dataset_column,overall_number,prevalence,MF_prevalence_ratio,FM_prevalence_ratio,overall_number_of_participants
0,D12_M,7533,118108,4,5,/cs/labs/michall/roeizucker/10krun/results/D12...,M,D12 Benign neoplasm of colon,['D12'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,11,125641,0.059957,1.647540,0.606965,125641
1,C50_M,77,125564,-1,-1,,M,C50 Malignant neoplasm of breast,['C50'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,8,125641,0.000613,0.008559,116.835251,125641
2,D64,11389,263435,0,4,/cs/labs/michall/roeizucker/10krun/results/D64...,B,D64 Other anaemias,['D64'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,12,274824,0.041441,0.908242,1.101029,274824
3,C44,18944,255880,47,65,/cs/labs/michall/roeizucker/10krun/results/C44...,B,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,3,274824,0.068931,1.315811,0.759987,274824
4,C44_F,8986,140197,8,18,/cs/labs/michall/roeizucker/10krun/results/C44...,F,C44 Other malignant neoplasms of skin,['C44'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/0:11/u...,4,149183,0.060235,1.315811,0.759987,149183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Z864_M,17992,107649,1,1,/cs/labs/michall/roeizucker/10krun/results/Z86...,M,Z86.4 Personal history of psychoactive substan...,['Z864'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/252:26...,8,125641,0.143202,-1.000000,-1.000000,125641
257,Z87_M,12601,113040,0,0,/cs/labs/michall/roeizucker/10krun/results/Z87...,M,Z87 Personal history of other diseases and con...,['Z87'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/252:26...,14,125641,0.100294,1.158148,0.863447,125641
258,Z867_F,7349,141834,0,3,/cs/labs/michall/roeizucker/10krun/results/Z86...,F,Z86.7 Personal history of diseases of the circ...,['Z867'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/252:26...,10,149183,0.049262,1.466891,0.681714,149183
259,Z864,30136,244688,0,0,/cs/labs/michall/roeizucker/10krun/results/Z86...,B,Z86.4 Personal history of psychoactive substan...,['Z864'],Binary,10krun,/cs/labs/michall/roeizucker/10krun/runs/252:26...,6,274824,0.109656,-1.000000,-1.000000,274824
